# ДЗ 3. Кластеризация методом DBScan


Данные описывают экономические условия в 48 городах мира в 1991 году.
Данные были собраны отделом экономических исследований банка Union
(Швейцария). Описаны экономические условия в 48 городах мира в 1991 году.  
Число наблюдений: 48
Названия переменных:  
City (Город): Название города  
Работа (Work): Взвешенное среднее числа рабочих часов, сосчитанное по 12 профессиям     
Цена (Price): Индекс цен 112 товаров и услуг, включая арендную плату за жилье(значение для Цюриха взято за 100%)    
Заработная плата (Salary): Индекс заработной платы за час работы, сосчитанный по 12 профессиям после налогов и вычетов (значение для Цюриха
взято за 100%)

In [221]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.cluster import DBSCAN

#  Планируем сравнить результаты кластеризации DBSCAN'ом
#  с результатом применения иерархического кластерного анализа
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster

In [222]:
df = pd.read_csv("Econom_Cities_data.csv", sep=";", index_col='City')
df.head(10)

,Work,Price,Salary
City,,,
Amsterdam,1714,"65,6",49
Athens,1792,"53,8","30,4"
Bogota,2152,"37,9","11,5"
Bombay,2052,"30,3","5,3"
Brussels,1708,"73,8","50,5"
Buenos_Aires,1971,"56,1","12,5"
Cairo,-9999,"37,1",-9999
Caracas,2041,61,"10,9"
Chicago,1924,"73,9","61,9"


### Предобработка данных

Необходимо заменить запятые на точки для численных операций. 

In [223]:
df['Price'] = df['Price'].str.replace(',', '.').astype(float)
df['Salary'] = df['Salary'].str.replace(',', '.').astype(float)
df.head(50)


,Work,Price,Salary
City,,,
Amsterdam,1714,65.6,49.0
Athens,1792,53.8,30.4
Bogota,2152,37.9,11.5
Bombay,2052,30.3,5.3
Brussels,1708,73.8,50.5
Buenos_Aires,1971,56.1,12.5
Cairo,-9999,37.1,-9999.0
Caracas,2041,61.0,10.9
Chicago,1924,73.9,61.9


В данной задаче необходима стандартизация, т.к. у признаков разные еденицы измерения.   
Приведем рабочие часы к относительному индексу (значение для Цюриха
взято за 100%)

In [224]:
df['Work'] = round(df['Work']/df.loc['Zurich', 'Work'] * 100, 1)
df.tail()

,Work,Price,Salary
City,,,
Tel_Aviv,107.9,67.3,27.0
Tokyo,100.6,115.0,68.0
Toronto,101.1,70.2,58.2
Vienna,95.3,78.0,51.3
Zurich,100.0,100.0,100.0


Города 'Cairo' и 'Jakarta' имеют некорректные значения. Их восстановление невозможно, поэтому удаляем соответствующие строки. 

In [225]:
df = df.drop(['Cairo', 'Jakarta'])

### Подбор параметров модели DBSCAN.
Начинаем со значений по умолчанию

In [226]:
#  Значения 3-х первых параметров совпадают со значениями "по умолчанию"
dbscan_1 = DBSCAN()
dbscan_1.fit(df)
dbscan_1.labels_ # Numpy массив номеров кластеров


array([-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1])

Результат очень плохой, все улетело в выбросы. Смотрим, какие значения использовались. Пытаемся увеличить eps. 

Значения параметров "по умолчанию":  
eps=0.5,   
metric='euclidean',   
min_samples=5

In [227]:
#  Создаем таблицу частот в numpy
#  Ту же самую таблицу

dbscan_1 = DBSCAN(eps=10, metric='euclidean', min_samples=5)
dbscan_1.fit(df)

unique, counts = np.unique(dbscan_1.labels_, return_counts=True)
print(np.asarray((unique, counts)).T)

[[-1 24]
 [ 0 13]
 [ 1  9]]


In [228]:
dbscan_1 = DBSCAN(eps=15, metric='euclidean', min_samples=5)
dbscan_1.fit(df)

unique, counts = np.unique(dbscan_1.labels_, return_counts=True)
print(np.asarray((unique, counts)).T)

[[-1  8]
 [ 0 19]
 [ 1 19]]


In [229]:
dbscan_1 = DBSCAN(eps=20, metric='euclidean', min_samples=5)
dbscan_1.fit(df)

unique, counts = np.unique(dbscan_1.labels_, return_counts=True)
print(np.asarray((unique, counts)).T)

[[-1  6]
 [ 0 40]]


In [230]:
dbscan_1 = DBSCAN(eps=14, metric='euclidean', min_samples=5)
dbscan_1.fit(df)

unique, counts = np.unique(dbscan_1.labels_, return_counts=True)
print(np.asarray((unique, counts)).T)

[[-1  9]
 [ 0 19]
 [ 1 18]]


In [231]:
dbscan_1 = DBSCAN(eps=19, metric='euclidean', min_samples=5)
dbscan_1.fit(df)

unique, counts = np.unique(dbscan_1.labels_, return_counts=True)
print(np.asarray((unique, counts)).T)

[[-1  8]
 [ 0 19]
 [ 1 19]]


Таким образом eps = 15 - наиболее оптимален. Теперь меняем min samples

In [232]:
dbscan_1 = DBSCAN(eps=15, metric='euclidean', min_samples=4)
dbscan_1.fit(df)

unique, counts = np.unique(dbscan_1.labels_, return_counts=True)
print(np.asarray((unique, counts)).T)

[[-1  8]
 [ 0 19]
 [ 1 19]]


In [233]:
dbscan_1 = DBSCAN(eps=15, metric='euclidean', min_samples=3)
dbscan_1.fit(df)

unique, counts = np.unique(dbscan_1.labels_, return_counts=True)
print(np.asarray((unique, counts)).T)

[[-1  5]
 [ 0 19]
 [ 1 19]
 [ 2  3]]


In [234]:
dbscan_1 = DBSCAN(eps=15, metric='euclidean', min_samples=2)
dbscan_1.fit(df)

unique, counts = np.unique(dbscan_1.labels_, return_counts=True)
print(np.asarray((unique, counts)).T)

[[-1  3]
 [ 0 19]
 [ 1 19]
 [ 2  2]
 [ 3  3]]


In [235]:
dbscan_1 = DBSCAN(eps=15, metric='euclidean', min_samples=10)
dbscan_1.fit(df)

unique, counts = np.unique(dbscan_1.labels_, return_counts=True)
print(np.asarray((unique, counts)).T)

[[-1 27]
 [ 0 19]]


При min_samples=2 - уменьшились выбросы, оставляем это значение. 

In [236]:
dbscan_1 = DBSCAN(eps=15, metric='euclidean', min_samples=2)
dbscan_1.fit(df)

unique, counts = np.unique(dbscan_1.labels_, return_counts=True)
print(np.asarray((unique, counts)).T)

[[-1  3]
 [ 0 19]
 [ 1 19]
 [ 2  2]
 [ 3  3]]


###  Сравнение с результатами иерархического кластерного анализа <br>

In [237]:
link = linkage(df, 'ward', 'euclidean')

Для сравнения кластеризации  dbscan_2 и dbscan_3 с результатами иерархического кластерного анализа строим таблицу сопряженности

In [238]:
# Создаю таблицу res_  с результатами разных кластеризаций

res_ = pd.DataFrame()

res_['dbscan_1'] = dbscan_1.labels_
res_['cluster'] = fcluster(link, 5, criterion='maxclust')


In [239]:
#  Таблица сопряженности для двух кластеризаций
tab = pd.crosstab(res_['dbscan_1'], res_['cluster'])

print(tab)


cluster    1  2   3  4  5
dbscan_1                 
-1         0  2   0  0  1
 0         0  0  17  0  2
 1        10  9   0  0  0
 2         0  0   0  2  0
 3         0  0   0  0  3


1 и 2 кластеры слились в один с небольшими выбросами, 3 и 4 сохранились, а 5 сохранился на половину, остальное расформировалось. 

In [240]:
df_2 = df.copy()

In [241]:
df_2['dbscan_1'] = dbscan_1.labels_
df_2['cluster'] = fcluster(link, 5, criterion='maxclust')

In [242]:
df_2.groupby('cluster').mean()

,Work,Price,Salary,dbscan_1
cluster,,,,
1,107.120000,43.610000,8.030000,1.000000
2,105.772727,60.381818,23.909091,0.636364
3,96.429412,75.758824,55.005882,0.000000
4,100.300000,97.950000,95.150000,2.000000
5,92.416667,106.750000,58.400000,1.333333


In [243]:
df.groupby(df_2['dbscan_1']).mean()

,Work,Price,Salary
dbscan_1,,,
-1,112.833333,86.466667,33.833333
0,95.931579,77.526316,55.157895
1,104.884211,50.115789,14.789474
2,100.300000,97.950000,95.150000
3,91.500000,114.700000,66.100000


Выбросы не интересуют.  \
Получили следующие группы:   \
0 - маленький рабочий день, цены умеренные, зарплаты большие \
1 - низкие зарплаты и низкие цены, самый долгий рабочий день \
2 - достаточно большой рабочий день, большие цены и большие зарплаты \
3 -  очень высокие цены, зарплата выше среднего и самый короткий рабочий день 

###  Silhouette для автоматического определения значений параметров

In [244]:
from sklearn import metrics

In [245]:
#  Создаем списки тех значений параметров, которые собираюся перебирать.
# (grid search)

eps_1 = [10, 10.5, 11, 11.5, 12, 12.5, 13, 13.5, 14, 14.5, 15, 15.5, 16, 16.5, 17, 18, 19, 20]
min_samples_1 = [2, 3, 4, 5, 6, 7]

In [246]:
sil_avg = []
#  Для сохранения наилучшено набора параметров
max_value = [0, 0, 0, -1]

In [247]:
#  Перебираем все пары значений параметров
#  Сохраняем лучшее решение

for i in range(len(eps_1)):
    for j in range(len(min_samples_1)):

        db = DBSCAN(min_samples = min_samples_1[j], eps =eps_1[i]).fit(df)
        labels = db.labels_

        # Число кластеров, после отбрасывания выбросов.
        n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)

        if n_clusters_ > 1:
            silhouette_avg = metrics.silhouette_score(df, labels)
            if silhouette_avg > max_value[3]:
                max_value=(eps_1[i], min_samples_1[j], n_clusters_, silhouette_avg)
            sil_avg.append(silhouette_avg)

print("epsilon=", max_value[0], 
      "\nmin_sample=", max_value[1],
      "\nnumber of clusters=", max_value[2],
      "\naverage silhouette score= %.4f" % max_value[3])

epsilon= 13.5 
min_sample= 2 
number of clusters= 4 
average silhouette score= 0.4972


In [250]:
dbscan_2 = DBSCAN(eps=max_value[0], metric='euclidean', min_samples=max_value[1], algorithm= 'brute')
dbscan_2.fit(df)

unique, counts = np.unique(dbscan_2.labels_, return_counts=True)
print(np.asarray((unique, counts)).T)

[[-1  3]
 [ 0 19]
 [ 1 19]
 [ 2  2]
 [ 3  3]]


Получили то же, что и вручную. 